# Complete documentation of excution of TWAS Rosmap MV Susie analysis
Complete workflow to generate multivariate susie TWAS result for the ROSMAP data set.

## Overview
1. Data Reformating
2. Univariate SuSiE
2. Estimation of whole genome prior
3. multivariate susie
4. Reformatting of mv SuSiE result
5. Association testing

## Data reformatting
This step transform the format of ROSMAP RNA-Seq data and annotate the posistion of each genes

In [ ]:
sos run /home/hs3163/GIT/neuro-twas/workflow/rosmap_reformatting.ipynb \
    --ref "/home/hs3163/Project/Rosmap/data/patient_key_WGS.txt" \
    --name_col 2 \
    --real_name_col 1 \
    --gene_exp "/mnt/mfs/ctcn/datasets/rosmap/rnaseq/dlpfcTissue/batch0-m5/values/residuals/geneTpmResidualsAgeGenderAdj.txt" \
    --start_at 2 \
    --output "/home/hs3163/Project/Rosmap/data/gene_exp/DLPFC" \
    -J 6 -q csg -c ~/system_file/csg.yml

sos run /home/hs3163/GIT/neuro-twas/workflow/rosmap_reformatting.ipynb \
    --ref "/home/hs3163/Project/Rosmap/data/patient_key_WGS.txt" \
    --name_col 2 \
    --real_name_col 1 \
    --gene_exp "/mnt/mfs/ctcn/datasets/rosmap/rnaseq/pccTissue/mbatch1-5/values/residuals/geneTpmResidualsAgeGenderAdj.txt" \
    --start_at 2 \
    --output "/home/hs3163/Project/Rosmap/data/gene_exp/PCC" \
    -J 6 -q csg -c ~/system_file/csg.yml

sos run /home/hs3163/GIT/neuro-twas/workflow/rosmap_reformatting.ipynb \
    --ref "/home/hs3163/Project/Rosmap/data/patient_key_WGS.txt" \
    --name_col 2 \
    --real_name_col 1 \
    --gene_exp "/mnt/mfs/ctcn/datasets/rosmap/rnaseq/acTissue/mbatch1-4/values/residuals/geneTpmResidualsAgeGenderAdj.txt" \
    --start_at 2 \
    --output "/home/hs3163/Project/Rosmap/data/gene_exp/AC" &

## Generation of Whole genome prior
with modified windows and shared SNPs

In [ ]:
nohup sos run ~/GIT/neuro-twas/workflow/uni_susie.ipynb susie  \
  --molecular-pheno  /home/hs3163/Project/Rosmap/data/gene_exp/AC/geneTpmResidualsAgeGenderAdj_rename.txt  \
  --wd /home/hs3163/Project/Genome_prior/AC \
  --genotype_list /home/hs3163/Project/Rosmap/data/Rosmap_wgs_genotype_list.txt \
  --region_list /home/hs3163/Project/Rosmap/data/gene_exp/AC/geneTpmResidualsAgeGenderAdj_rename_region_list.txt \
  --region_name 1 \
  --data_start 2 \
  --window 500000 \
  --container /mnt/mfs/statgen/containers/twas_latest.sif \
  --job_size 1\
  -J 50 -q csg -c /mnt/mfs/statgen/pbs_template/csg.yml -s build &

In [ ]:
nohup sos run ~/GIT/neuro-twas/workflow/uni_susie.ipynb susie  \
  --molecular-pheno  /home/hs3163/Project/Rosmap/data/gene_exp/PCC/geneTpmResidualsAgeGenderAdj_rename.txt  \
  --wd /home/hs3163/Project/Genome_prior/PCC \
  --genotype_list /home/hs3163/Project/Rosmap/data/Rosmap_wgs_genotype_list.txt \
  --region_list /home/hs3163/Project/Rosmap/data/gene_exp/PCC/geneTpmResidualsAgeGenderAdj_rename_region_list.txt \
  --region_name 1 \
  --data_start 2 \
  --window 500000 \
  --container /mnt/mfs/statgen/containers/twas_latest.sif \
  --job_size 1\
  -J 50 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &

In [ ]:
nohup sos run /home/hs3163/GIT/neuro-twas/workflow/uni_susie.ipynb susie  \
  --molecular-pheno  /home/hs3163/Project/Rosmap/data/gene_exp/DLPFC/geneTpmResidualsAgeGenderAdj_rename.txt  \
  --wd /home/hs3163/Project/Genome_prior/DLPFC \
  --genotype_list /home/hs3163/Project/Rosmap/data/Rosmap_wgs_genotype_list.txt \
  --region_list /home/hs3163/Project/Rosmap/data/gene_exp/DLPFC/geneTpmResidualsAgeGenderAdj_rename_region_list.txt \
  --region_name 1 \
  --data_start 2 \
  --window 500000 \
  --container /mnt/mfs/statgen/containers/twas_latest.sif \
  --job_size 1\
  -J 50 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &

### Compute the mixture prior ( Use the updated version)

#### Merging the tissues

In [ ]:
nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb merge \
--molecular_pheno /home/hs3163/Project/Genome_prior/data/molc_dir    \
--analysis-units /home/hs3163/Project/Genome_prior/data/rds_list  \
--cwd   /home/hs3163/Project/Genome_prior/mixture_prior/output \
--name "geneTpmResidualsAgeGenderAdj_rename" \
--container /mnt/mfs/statgen/containers/twas_latest.sif \
-s build &

#### Extracting the strongest, null .etc signals

In [ ]:
    cd /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/  && ls *.rds | sed 's/\.rds//g' > analysis_units.txt && cd -
    nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb extract_effects \
        --analysis-units /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/analysis_units.txt \
        --datadir /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/ \
        --name 'susie'  \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --cwd /home/hs3163/Project/Genome_prior/mixture_prior/output/ &

#### Create flashr output


In [ ]:
nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb flash \
        --analysis-units /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/analysis_units.txt \
        --datadir /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/ \
        --name 'susie'  \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --cwd /home/hs3163/Project/Genome_prior/mixture_prior/output/ &  \
        -J 50 -q csg -c /home/hs3163/GIT/neuro-twas/code/csg.yml -s build &
        
nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb flash_nonneg \
        --analysis-units /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/analysis_units.txt \
        --datadir /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/ \
        --name 'susie'  \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --cwd /home/hs3163/Project/Genome_prior/mixture_prior/output/ \
        -J 50 -q csg -c /home/hs3163/GIT/neuro-twas/code/csg.yml -s build &

nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb pca \
        --analysis-units /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/analysis_units.txt \
        --datadir /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/ \
        --name 'susie'  \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --cwd /home/hs3163/Project/Genome_prior/mixture_prior/output/ \
        -J 50 -q csg -c /home/hs3163/GIT/neuro-twas/code/csg.yml -s build &
        
nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb canonical \
        --analysis-units /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/analysis_units.txt \
        --datadir /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/ \
        --name 'susie'  \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --cwd /home/hs3163/Project/Genome_prior/mixture_prior/output/ \
        -J 50 -q csg -c /home/hs3163/GIT/neuro-twas/code/csg.yml -s build &


#### Extreme Deconvolution

In [ ]:
nohup sos dryrun ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb ud \
        --analysis-units /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/analysis_units.txt \
        --datadir /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/ \
        --name 'susie'  \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --cwd /home/hs3163/Project/Genome_prior/mixture_prior/output/ \
        --scale_only  "canonical" &
        \
        -J 50 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &

nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb ud --ud-method teem  \
        --analysis-units /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/analysis_units.txt  \
        --datadir /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/ \
        --name 'susie'  \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --cwd /home/hs3163/Project/Genome_prior/mixture_prior/output/ \
        --scale_only  "/home/hs3163/Project/Genome_prior/mixture_prior/output/susie.canonical.rds" &
        \
        -J 50 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &
        
nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb ed \
        --analysis-units /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/analysis_units.txt \
        --datadir /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/ \
        --name 'susie'  \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --cwd /home/hs3163/Project/Genome_prior/mixture_prior/output/ &
        \
        -J 50 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &



In [ ]:
nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb plot_U \
        --model_data /home/hs3163/Project/Genome_prior/mixture_prior/output/susie.ed.rds \
        --name ""  \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --tol "0.06"  \
        --cwd /home/hs3163/Project/Genome_prior/mixture_prior/output/ \
        -J 50 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &

## MV SuSiE
### Extracting Overlapped SNPs

In [ ]:
nohup sos run ~/GIT/neuro-twas/workflow/mv_susie.ipynb snp_exclude \
--molecular_pheno_dir "/mnt/mfs/statgen/neuro-twas/mv_wg/molecular_phenotype_list" \
--region_list /mnt/mfs/statgen/neuro-twas/mv_wg/wg_rds_list_final \
--wd "/mnt/mfs/statgen/neuro-twas/whole_genome_0504/" \
--name_prefix "geneTpmResidualsAgeGenderAdj_rename" \
--container /mnt/mfs/statgen/containers/twas_latest.sif \
--impute TRUE --job_size 1 \
--cv_times 100 \
--mixture_prior '/home/hs3163/Project/Genome_prior/mixture_prior/output/susie.ed.rds' \
--extract_snp "~/Project/Alz/data/AD_sumstats_SNP" \
-J 200 -q csg -c ~/GIT/neuro-twas/code/csg.yml -s build &

### Removing the genes whithout overlapping snvs

In [ ]:
bash: cd /mnt/mfs/statgen/neuro-twas/whole_genome_0504/cache
      ls *.bed > keep
R:
    library(readr)
    library(dplyr)
    library(tibble)
    list = read.table("./keep",sep = ".")
    gene_list = read_delim("/mnt/mfs/statgen/neuro-twas/mv_wg/wg_rds_list_final",delim = "\t")
    gene_list = gene_list%>%filter(`#region` %in% list$V2 )
    gene_list%>%write_delim("wg_rds_list_final",delim = "\t")    

### Weight estimation With three types of mixture prior

In [ ]:
nohup sos run /home/hs3163/GIT/neuro-twas/workflow/mv_susie.ipynb mv_susie \
--molecular_pheno_dir "/mnt/mfs/statgen/neuro-twas/mv_wg/molecular_phenotype_list" \
--region_list /mnt/mfs/statgen/neuro-twas/whole_genome_0504/wg_rds_list_final \
--wd "/mnt/mfs/statgen/neuro-twas/whole_genome_0504/" \
--name_prefix "geneTpmResidualsAgeGenderAdj_rename" \
--container /mnt/mfs/statgen/containers/twas_latest.sif \
--impute TRUE --job_size 1 \
--cv_times 100 \
--mixture_prior '/home/hs3163/Project/Genome_prior/mixture_prior/output/susie.teem.rds' \
--extract_snp "~/Project/Alz/data/AD_sumstats_SNP" \
-J 200 -q csg -c /home/hs3163/GIT/neuro-twas/code/csg.yml -s build &

nohup sos run ~/GIT/neuro-twas/Workflow/mv_susie.ipynb mv_susie \
--molecular_pheno_dir "/mnt/mfs/statgen/neuro-twas/mv_wg/molecular_phenotype_list" \
--region_list /mnt/mfs/statgen/neuro-twas/mv_wg/wg_rds_list_final \
--wd "/mnt/mfs/statgen/neuro-twas/whole_genome_0504/" \
--name_prefix "geneTpmResidualsAgeGenderAdj_rename" \
--container /mnt/mfs/statgen/containers/twas_latest.sif \
--impute TRUE --job_size 1 \
--cv_times 100 \
--mixture_prior '/home/hs3163/Project/Genome_prior/mixture_prior/output/susie.ed_bovy.rds' \
-J 200 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &

nohup sos run ~/GIT/neuro-twas/Workflow/mv_susie.ipynb mv_susie \
--molecular_pheno_dir "/mnt/mfs/statgen/neuro-twas/mv_wg/molecular_phenotype_list" \
--region_list /mnt/mfs/statgen/neuro-twas/mv_wg/wg_rds_list_final \
--wd "/mnt/mfs/statgen/neuro-twas/whole_genome_0504/" \
--name_prefix "geneTpmResidualsAgeGenderAdj_rename" \
--container /mnt/mfs/statgen/containers/twas_latest.sif \
--impute TRUE --job_size 1 \
--cv_times 100 \
--mixture_prior '/home/hs3163/Project/Genome_prior/mixture_prior/output/susie.teem.rds' \
-J 200 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &

### Output Reformatting

In [ ]:
nohup sos run /home/hs3163/GIT/neuro-twas/workflow/mv_susie.ipynb fusion_tf \
--molecular_pheno_dir /mnt/mfs/statgen/neuro-twas/mv_wg/molecular_phenotype_list   --region_list /mnt/mfs/statgen/neuro-twas/whole_genome_0504/wg_rds_list_final \
--wd /mnt/mfs/statgen/neuro-twas/mv_wg/wg_rds_list_final \
--name_prefix "geneTpmResidualsAgeGenderAdj_rename" \
--container /mnt/mfs/statgen/containers/twas_latest.sif \
--extract_snp "~/Project/Alz/data/AD_sumstats_SNP" \
--impute TRUE --cv_times 2  &



nohup sos dryrun /home/hs3163/GIT/neuro-twas/workflow/mv_susie.ipynb fusion_tf \
--molecular_pheno_dir /mnt/mfs/statgen/neuro-twas/mv_wg/molecular_phenotype_list   \
--region_list /home/hs3163/Project/mwe/mv_susie_mwe/region_list \
--wd /mnt/mfs/statgen/neuro-twas/mv_wg/wg_rds_list_final \
--name_prefix "geneTpmResidualsAgeGenderAdj_rename" \
--container /mnt/mfs/statgen/containers/twas_latest.sif \
--extract_snp "~/Project/Alz/data/AD_sumstats_SNP" \
--impute TRUE --cv_times 2  &



nohup sos run /home/hs3163/GIT/neuro-twas/workflow/mv_susie.ipynb fusion_tf \
--molecular_pheno_dir /mnt/mfs/statgen/neuro-twas/whole_genome_0504/molecular_phenotype_list \
--region_list /mnt/mfs/statgen/neuro-twas/whole_genome_0504/wg_rds_list_test \
--wd /mnt/mfs/statgen/neuro-twas/whole_genome_0504/ \
--name_prefix "geneTpmResidualsAgeGenderAdj_rename" \
--container /mnt/mfs/statgen/containers/twas_latest.sif \
--extract_snp "~/Project/Alz/data/AD_sumstats_SNP" --impute TRUE \
--cv_times 2 &



## Association testing

In [ ]:
wd="whole_genome_0504"

nohup sos run /home/hs3163/GIT/neuro-twas/workflow/twas_fusion.ipynb association_test  \
  --gwas_sumstat /home/hs3163/Project/Alz/data/AD_sumstats_Jansenetal_2019sept.txt \
  --molecular-pheno /home/hs3163/Project/Rosmap/data/gene_exp/DLPFC/geneTpmResidualsAgeGenderAdj_rename.txt \
  --wd /mnt/mfs/statgen/neuro-twas/$wd/wgt/DLPFC \
  --genotype_list /home/hs3163/Project/Rosmap/data/Rosmap_wgs_genotype_list.txt \
  --region_list /home/hs3163/Project/mwe/mv_susie_mwe/region_list \
  --region_name 1 \
  --data_start 2 \
  --window 500000 \
  --weights_path /mnt/mfs/statgen/neuro-twas/$wd/wgt/DLPFC \
  --weights_list /mnt/mfs/statgen/neuro-twas/$wd/wgt/DLPFC/All_wgt_list.txt \
  --container /mnt/mfs/statgen/containers/twas_latest.sif \
  --job_size 1\
  --max_impute 1\
  --asso_mem "50G" \
  -J 40 -q csg -c /home/hs3163/GIT/neuro-twas/code/csg.yml &
  -s build &

nohup sos run ~/GIT/neuro-twas/workflow/twas_fusion.ipynb association_test  \
  --gwas_sumstat /home/hs3163/Project/Alz/data/AD_sumstats_Jansenetal_2019sept.txt \
  --molecular-pheno /home/hs3163/Project/Rosmap/data/gene_exp/DLPFC/geneTpmResidualsAgeGenderAdj_rename.txt \
  --wd /mnt/mfs/statgen/neuro-twas/$wd/wgt/PCC \
  --genotype_list /home/hs3163/Project/Rosmap/data/Rosmap_wgs_genotype_list.txt \
  --region_list /home/hs3163/Project/mwe/mv_susie_mwe/region_list \
  --region_name 1 \
  --data_start 2 \
  --window 500000 \
  --weights_path /mnt/mfs/statgen/neuro-twas/$wd/wgt/PCC \
  --weights_list /mnt/mfs/statgen/neuro-twas/$wd/wgt/PCC/All_wgt_list.txt \
  --container /mnt/mfs/statgen/containers/twas_latest.sif \
  --job_size 1\
  --max_impute 1\
  --asso_mem "50G" \
  -J 40 -q csg -c /home/hs3163/GIT/neuro-twas/code/csg.yml \
  -s build &

nohup sos run /home/hs3163/GIT/neuro-twas/workflow/twas_fusion.ipynb association_test  \
  --gwas_sumstat /home/hs3163/Project/Alz/data/AD_sumstats_Jansenetal_2019sept.txt \
  --molecular-pheno /home/hs3163/Project/Rosmap/data/gene_exp/DLPFC/geneTpmResidualsAgeGenderAdj_rename.txt \
  --wd /mnt/mfs/statgen/neuro-twas/$wd/wgt/AC \
  --genotype_list /home/hs3163/Project/Rosmap/data/Rosmap_wgs_genotype_list.txt \
  --region_list /home/hs3163/Project/mwe/mv_susie_mwe/region_list \
  --region_name 1 \
  --data_start 2 \
  --window 500000 \
  --weights_path /mnt/mfs/statgen/neuro-twas/$wd/wgt/AC \
  --weights_list /mnt/mfs/statgen/neuro-twas/$wd/wgt/AC/All_wgt_list.txt \
  --container /mnt/mfs/statgen/containers/twas_latest.sif \
  --job_size 1\
  --max_impute 1\
  --asso_mem "50G" \
  -J 40 -q csg -c /home/hs3163/GIT/neuro-twas/code/csg.yml \
  -s build &